In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
from sklearn import linear_model
# from scipy import stats
import seaborn as sns
import altair as alt
import plotly.express as px

## merge promice data to one dataframe

In [ ]:
df = pd.read_csv('promice/promice.csv')
df['Longitude'] = df['Longitude'] * -1

In [ ]:
folderpath = "promice/modis500m"

searchCriteria = "*.csv"
globInput = os.path.join(folderpath, searchCriteria)
csvPath = glob.glob(globInput)
csvList = os.listdir(folderpath)

In [ ]:
# daily  
for i in range(len(csvList)):
    # promice data
    stationName = os.path.splitext(csvList[i])[0].replace("-", "*")
    index = df.index[df.Station == stationName][0]
    url = df.url[index] # daily
    # url = df.urlhourly[index]
    dfs = pd.read_table(url, sep=r'\s{1,}', engine='python')

    dfs = dfs[['Albedo_theta<70d', 'LatitudeGPS_HDOP<1(degN)', 'LongitudeGPS_HDOP<1(degW)', 'Year', 'MonthOfYear', 'DayOfMonth','CloudCover']]
    dfs = dfs.replace(-999, np.nan)
    dfs['lon'] = dfs['LongitudeGPS_HDOP<1(degW)'].interpolate(method='linear',limit_direction='both') * -1
    dfs['lat'] = dfs['LatitudeGPS_HDOP<1(degN)'].interpolate(method='linear',limit_direction='both')
    dfs['datetime'] = pd.to_datetime(dict(year=dfs.Year, month=dfs.MonthOfYear, day = dfs.DayOfMonth))
    # cloud cover less than 50% and albedo must be valid value
    dfs = dfs[(dfs['Albedo_theta<70d'] > 0) & (dfs['CloudCover'] < 0.5)]
    dfs['Station'] = stationName

    # satellite data
    dfr = pd.read_csv(csvPath[i])
    dfr['Snow_Albedo_Daily_Tile'] = dfr['Snow_Albedo_Daily_Tile'] / 100
    # dfr.datetime = pd.to_datetime(dfr.datetime).dt.date # keep only ymd
    dfr.datetime = pd.to_datetime(dfr.datetime)

    # join by datetime
    # dfmerge = pd.merge(dfr, dfs, how='outer', on='datetime')
    dfmerge = pd.merge_asof(dfr.sort_values('datetime'), dfs, on='datetime',allow_exact_matches=False, tolerance=pd.Timedelta(days=1) )
    if i==0:
        dfmerge.dropna().to_csv('promice vs satellite modis.csv', mode='w', index=False)
    else:
        dfmerge.dropna().to_csv('promice vs satellite modis.csv', mode='a', index=False, header=False)

## Lienar Regression: PROMICE VS MODIS albedo

In [ ]:
df = pd.read_csv("promice vs satellite modis.csv")
# ProfileReport(df)
df = df[(df['MonthOfYear']>4) & (df['MonthOfYear']<10)] # (df['MonthOfYear']!=7
# df = df[df['Albedo_theta<70d']<0.9]
# df[['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2']] = df[['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2']] / 10000

In [ ]:
boxplot = df.boxplot(column=['Snow_Albedo_Daily_Tile', 'Albedo_theta<70d'])

In [ ]:
df.Station.value_counts().plot(kind='bar')

In [ ]:

X = df[['Snow_Albedo_Daily_Tile']] 
# X = df[['Blue', 'Green', 'NIR', 'SWIR1', 'SWIR2']] 
y = df['Albedo_theta<70d'] 

# mask = df['MonthOfYear']>6 
# y[mask] = y[mask]/1.1

In [ ]:
ols = linear_model.LinearRegression()
model = ols.fit(X, y)
response = model.predict(X)
r2 = model.score(X, y)

In [ ]:
print('R\N{SUPERSCRIPT TWO}: %.4f' % r2)
print(model.coef_)
# print("coefficients: Blue: %.4f, Green: %.4f, Red: %.4f, NIR: %.4f, SWIR1: %.4f, SWIR2: %.4f" %(model.coef_[0], model.coef_[1], model.coef_[2], model.coef_[3], model.coef_[4], model.coef_[5]))
# print("coefficients: Blue: %.4f, Red: %.4f, NIR: %.4f, SWIR1: %.4f, SWIR2: %.4f" %(model.coef_[0], model.coef_[1], model.coef_[2], model.coef_[3], model.coef_[4]))
print("intercept: %.4f" % model.intercept_)
len(df)

In [ ]:

fig, ax = plt.subplots(figsize=(8, 8))
# plt.sca(ax1)


sns.set_theme(style="darkgrid")
sns.scatterplot(x=response, y=y, s=10 )
sns.regplot(x=response, y=y, scatter=False, color='red',)
SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 20
plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=BIGGER_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.rcParams["font.family"] = "Arial"
# plt.plot([0,1], [0,1], color = 'white') # reference line
plt.xlim(0, 1)
plt.ylim(0, 1)
ax.set(xlabel='Predicted Albedo (MODIS)', ylabel='Albedo PROMICE')
ax.set_aspect('equal', 'box')
# sns.histplot(x=response, y=y, bins=50, pthresh=.1, cmap="viridis", cbar=True, cbar_kws={'label': 'frequency'})
# sns.kdeplot(x=response, y=y, levels=5, color="w", linewidths=1)
fig.savefig("print/MODISalbedoPromice.png", dpi=300, bbox_inches="tight")

In [ ]:
df['response'] = response
alt.data_transformers.disable_max_rows() # this should be avoided but now let's disable the limit
alt.Chart(df).mark_circle().encode(
    x='response',
    y='Albedo_theta<70d',
    color='Station',
    tooltip=['datetime:T','Station','response','Albedo_theta<70d']
).interactive()

# chart + chart.transform_regression('x', 'y').mark_line()

In [ ]:
df['response'] = response
alt.data_transformers.disable_max_rows() # this should be avoided but now let's disable the limit

brush = alt.selection(type='interval')
points = alt.Chart(df).mark_circle().encode(
    x='response',
    y='Albedo_theta<70d',
    color=alt.condition(brush, 'Station:O', alt.value('grey')),
    tooltip=['datetime:T','Station','response','Albedo_theta<70d']
).add_selection(brush)
# Base chart for data tables
ranked_text = alt.Chart(df).mark_text().encode(
    y=alt.Y('row_number:O',axis=None)
).transform_window(
    row_number='row_number()'
).transform_filter(
    brush
).transform_window(
    rank='rank(row_number)'
).transform_filter(
    alt.datum.rank<40
)

# Data Tables
stationalt = ranked_text.encode(text='Station').properties(title='station')
albedoalt = ranked_text.encode(text='Albedo_theta<70d:N').properties(title='Albedo')
predictedalt = ranked_text.encode(text='response:N').properties(title='predicted albedo')
timealt = ranked_text.encode(text='datetime:T').properties(title='time')
text = alt.hconcat(stationalt, albedoalt, predictedalt, timealt) # Combine data tables

# Build chart
alt.hconcat(
    points,
    text
).resolve_legend(
    color="independent"
)
# chart + chart.transform_regression('x', 'y').mark_line()